<a href="https://colab.research.google.com/github/CAU2022-CAPSTONE-PACETIME/BreathDetector/blob/BreathClassifier/BreathClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/ColabNotebooks

/content/drive/MyDrive/ColabNotebooks


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from imu import *
from BreathDataset import *
from torch.utils.data import DataLoader, random_split

BATCH_SIZE = 4
EPOCHS = 6 # 반복 횟수|

In [4]:
# input : mfcc(40, 44) --> (Batchsize, channel, h, w) --> (BATCH_SIZE, 1, 40, 44)
class BreathClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels = 1,
            out_channels = 4,
            kernel_size = (3, 3),
            stride = 1,
            padding = 1
        ),
        nn.BatchNorm2d(4),
        nn.ReLU()
    )
 
    self.conv2 = nn.Sequential(
        nn.Conv2d(
            in_channels = 4,
            out_channels = 4,
            kernel_size = (3, 3),
            stride = (1, 1),
            padding = 1
        ),
        nn.BatchNorm2d(4),
        nn.ReLU()
    )
    

    self.fc1 = nn.Sequential(
        nn.Linear(4*40*44, 500),
        nn.ReLU()
    )
    
    self.fc2 = nn.Sequential(
        nn.Linear(500, 200),
        nn.ReLU()
    )

    self.fc3 = nn.Sequential(
        nn.Linear(200, 50),
        nn.ReLU()
    )

    self.fc4 = nn.Sequential(
        nn.Linear(50, 1),
        nn.Sigmoid()
    )

    self.dropout = nn.Dropout(0.4)
  
  def forward(self, input):
    x = self.conv1(input)
    x = self.dropout(x)
    x = self.conv2(x)
    x = self.dropout(x)
    x = x.view(x.size(0), -1)
    
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)

    return x

In [5]:
def create_data_loader(train_data, batch_size):
  data_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, drop_last = True)
  return data_loader

def train_single_epoch(model, data_loader, loss_fn, optimizer, device):
  for input, target in data_loader:
    input = input.view(60, -1, 40, 44) # sound : batch*60x40x44->60*batch*40*44 imu : batch*60 -> 60*batch
    target = target.view(60, -1) 

    for i in range(len(input)):
      sound = input[i].view(-1, 1, 40, 44).to(device)
      imu = target[i].view(-1, 1).to(device)
      prediction = model(sound)
      loss = loss_fn(prediction, imu)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

def train(model, data_loader, loss_fn, optimizer, device, epochs):
  for i in range(epochs):
    print(f"Epoch {i + 1}")
    train_single_epoch(model, data_loader, loss_fn, optimizer, device)
    print("------------------------------")
  print("Finished training ")


def test(model, data_loader):
  with torch.no_grad():
    for input, target in data_loader:
      input = input.view(60, -1, 40, 44) # sound : batch*60x40x44->60*batch*40*44 imu : batch*60 -> 60*batch
      target = target.view(60, -1) 
        #print("input shape : {} target shape {}".format(input.shape, target.shape))
      accuracy = 0
      total = 0
      for i in range(len(input)):
        sound = input[i].view(-1, 1, 40, 44).to(device)
        imu = target[i].view(-1, 1).to(device)
   
        prediction = 1 if model(sound) >= 0.5 else 0
        total += 1
        accuracy += (prediction == imu).sum().item()
    print("accuracy : {}".format(100*accuracy/total))

In [6]:
if __name__ == "__main__":
  audio_list = ["/content/drive/MyDrive/ColabNotebooks/Data/note9-budslive", "/content/drive/MyDrive/ColabNotebooks/Data/A21s-airpodspro"]
  device = "cuda" if torch.cuda.is_available() else "cpu"
  dataset = BreathDataset(audio_list, device)
  print("Data length : {} Device : {}".format(len(dataset), device))

  train_ratio = 0.8
  train_data_length = int(train_ratio * len(dataset))
  test_data_length = len(dataset) - train_data_length

  train_dataset, test_dataset = random_split(dataset, [train_data_length, test_data_length])
  train_data_loader = create_data_loader(train_dataset, BATCH_SIZE)
  test_data_loader = create_data_loader(test_dataset, 1)
  
  cnn = BreathClassifier().to(device)
  print(cnn)

  loss_fn = nn.BCELoss()
  optimizer = optim.Adam(cnn.parameters(), lr = 0.001)

  train(cnn, train_data_loader, loss_fn, optimizer, device, EPOCHS)
  torch.save(cnn.state_dict(), "BreathClassifier.pth")


Data length : 63 Device : cuda
BreathClassifier(
  (conv1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Linear(in_features=7040, out_features=500, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=500, out_features=200, bias=True)
    (1): ReLU()
  )
  (fc3): Sequential(
    (0): Linear(in_features=200, out_features=50, bias=True)
    (1): ReLU()
  )
  (fc4): Sequential(
    (0): Linear(in_features=50, out_features=1, bias=True)
    (1): Sigmoid()
  )
  (dropout): Dropout(p=0.4, inplace=False)
)
Epoch 1


/content/drive/MyDrive/ColabNotebooks/BreathDataset.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  return torch.FloatTensor(mfcc_list), torch.FloatTensor(imu.get_item())


------------------------------
Epoch 2
------------------------------
Epoch 3
------------------------------
Epoch 4
------------------------------
Epoch 5
------------------------------
Epoch 6
------------------------------
Finished training 


In [8]:
# 모델 불러오기
model = BreathClassifier().to(device)
model.load_state_dict(torch.load("BreathClassifier.pth", map_location = device))
print(model)
test(model, test_data_loader)

BreathClassifier(
  (conv1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Linear(in_features=7040, out_features=500, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=500, out_features=200, bias=True)
    (1): ReLU()
  )
  (fc3): Sequential(
    (0): Linear(in_features=200, out_features=50, bias=True)
    (1): ReLU()
  )
  (fc4): Sequential(
    (0): Linear(in_features=50, out_features=1, bias=True)
    (1): Sigmoid()
  )
  (dropout): Dropout(p=0.4, inplace=False)
)
accuracy : 83.33333333333333


In [ ]:
import pandas as pd

def sound_process(path, index):
  data = pd.read_csv(path)
